importing required libraries from tensorflow

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import glob

In [0]:
image_size= [224,224]+[3]

[224,224] cause vgg takes input in that size only ang [3] is color format.

Now we will use pretrained model Densenet for Transfer Learning.


In [19]:
densenet = DenseNet201(input_shape=image_size, weights= 'imagenet',include_top=False)

74842112/74836368 [==============================] - 2s 0us/step


In [0]:
for layers in densenet.layers:
  layers.trainable= False

In [0]:
x = GlobalAveragePooling2D()(densenet.output)
x = Dropout(0.25)(x)

In [0]:
pred= Dense(units=2,activation='softmax')(x)

In [0]:
model= Model(inputs= densenet.input,outputs= pred)

In [0]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [25]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
______________________________________________________________________________________________

Data Augmentation cause dataset is small for deep learning purpose

In [0]:
train_datagen= ImageDataGenerator(rescale= 1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [27]:
training_set = train_datagen.flow_from_directory('/content/drive/My Drive/braintrain', target_size=(224,224),batch_size=32,class_mode='categorical',shuffle=True)

Found 210 images belonging to 2 classes.


In [0]:
test_datagen= ImageDataGenerator(rescale=1./255)

In [29]:
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/braintest', target_size=(224,224),batch_size=32,class_mode='categorical')

Found 43 images belonging to 2 classes.


Fitting the model to the dataset

In [30]:
model.fit(training_set,validation_data=test_set,epochs=10,steps_per_epoch=len(training_set),validation_steps=len(test_set),
                       callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)])

Epoch 1/10
7/7 [==============================] - 12s 2s/step - loss: 0.8493 - acc: 0.5714 - val_loss: 0.9287 - val_acc: 0.4884
Epoch 2/10
7/7 [==============================] - 2s 261ms/step - loss: 0.6515 - acc: 0.6190 - val_loss: 0.6864 - val_acc: 0.6047
Epoch 3/10
7/7 [==============================] - 3s 474ms/step - loss: 0.5500 - acc: 0.7143 - val_loss: 0.6512 - val_acc: 0.6512
Epoch 4/10
7/7 [==============================] - 3s 485ms/step - loss: 0.4435 - acc: 0.8000 - val_loss: 0.4627 - val_acc: 0.7442
Epoch 5/10
7/7 [==============================] - 3s 494ms/step - loss: 0.3975 - acc: 0.8571 - val_loss: 0.4611 - val_acc: 0.7907
Epoch 6/10
7/7 [==============================] - 3s 475ms/step - loss: 0.3741 - acc: 0.8476 - val_loss: 0.3499 - val_acc: 0.7907
Epoch 7/10
7/7 [==============================] - 3s 478ms/step - loss: 0.3226 - acc: 0.8905 - val_loss: 0.4103 - val_acc: 0.8372
Epoch 8/10
7/7 [==============================] - 3s 480ms/step - loss: 0.3104 - acc: 0.8667

Model Accuracy = 90.45 %

Now Saving the Model and Converting into tflite format for deployment

In [0]:
model.save("/content/drive/My Drive/brainmodel.h5")

In [32]:
converter = tf.lite.TFLiteConverter.from_keras_model_file('/content/drive/My Drive/brainmodel.h5')
modeltflite = converter.convert()
with open('/content/drive/My Drive/tumordetection.tflite', "wb") as f:
    f.write(modeltflite)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Froze 1006 variables.
INFO:tensorflow:Converted 1006 variables to const ops.


Tflite file size is 69.1Mb which is quite large

In next step I will apply parsing and Optimization for decresing file size for deployment